In [ ]:
!pip install iterative-stratification


In [ ]:
import pandas as pd
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
if __name__ == "__main__":
    TARGET_PATH = "../input/lish-moa/train_targets_scored.csv"
    df = pd.read_csv(TARGET_PATH)
    df.loc[:, 'kfold'] = -1
    targets = df.drop("sig_id", axis=1).values

    mskf = MultilabelStratifiedKFold(n_splits=5)

    for fold, (train_idx, valid_idx) in enumerate(mskf.split(X=df, y=targets)):
        df.loc[valid_idx, 'kfold'] = fold

    df.to_csv("train_targets_fold.csv", index=False)

In [ ]:
import torch
import torch.nn as nn

class MoaDataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, item):
        return {
            "x" : torch.tensor(self.features[item, :], dtype=torch.float32),
            "y": torch.tensor(self.targets[item, :], dtype=torch.float32),
        }

# Engine class is use for evaluating and training
class Engine:
    def __init__(self, model, optimizer, device):
        self.model = model
        self.device = device
        self.optimizer = optimizer

    @staticmethod
    def loss_fn(targets, outputs):
        return nn.BCEWithLogitsLoss()(outputs, targets)

    # Training functions
    def train(self, data_loader):
        self.model.train()
        final_loss = 0
        for data in data_loader:
            self.optimizer.zero_grad()
            inputs = data["x"].to(self.device)
            targets = data["y"].to(self.device)
            outputs = self.model(inputs)
            loss = self.loss_fn(targets, outputs)
            loss.backward()
            self.optimizer.step()
            final_loss += loss.item()
        return final_loss / len(data_loader)

    # Evaluating function
    def evaluate(self, data_loader):
        self.model.eval()
        final_loss = 0
        for data in data_loader:
            inputs = data["x"].to(self.device)
            targets = data["y"].to(self.device)
            outputs = self.model(inputs)
            loss = self.loss_fn(targets, outputs)
            final_loss += loss.item()
        return final_loss / len(data_loader)

class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size, num_layers, dropout):
        super().__init__()
        layers = []
        for _ in range(num_layers):
            if len(layers) == 0:
                layers.append(nn.Linear(num_features, hidden_size))
                layers.append(nn.BatchNorm1d(hidden_size))
                layers.append(nn.Dropout(dropout))
                layers.append(nn.Linear(hidden_size, hidden_size))
                layers.append(nn.BatchNorm1d(hidden_size))
                layers.append(nn.Dropout(dropout))
                layers.append(nn.ReLU())
            else:
                layers.append(nn.Linear(hidden_size, hidden_size))
                layers.append(nn.BatchNorm1d(hidden_size))
                layers.append(nn.Dropout(dropout))
                
                layers.append(nn.Linear(hidden_size, hidden_size))
                layers.append(nn.BatchNorm1d(hidden_size))
                layers.append(nn.Dropout(dropout))
                layers.append(nn.ReLU())

        layers.append(nn.Linear(hidden_size, num_targets))
        self.model = nn.Sequential(*layers)   # Layers in the list

    def forward(self, x):
        return self.model(x)





In [ ]:
import torch
import pandas as pd
import numpy as np
import optuna

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EPOCS = 100


def run_training(fold, params, save_model=False):
    df = pd.read_csv('../input/lish-moa/train_features.csv')
    df = df.drop(["cp_type", "cp_time", "cp_dose"], axis=1)
    targets_df = pd.read_csv("./train_targets_fold.csv")

    feature_cols = df.drop("sig_id", axis=1).columns
    target_cols = targets_df.drop(["sig_id", "kfold"], axis=1).columns

    df = df.merge(targets_df, on="sig_id", how="left")

    train_df = df[df.kfold != fold].reset_index(drop=True)
    valid_df = df[df.kfold == fold].reset_index(drop=True)

    xtrain = train_df[feature_cols].to_numpy()
    ytrain = train_df[target_cols].to_numpy()

    xvalid = valid_df[feature_cols].to_numpy()
    yvalid = valid_df[target_cols].to_numpy()

    train_dataset = MoaDataset(features=xtrain, targets=ytrain)
    valid_dataset = MoaDataset(features=xvalid, targets=yvalid)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=1024, num_workers=8, shuffle=True
    )

    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=1024, num_workers=8
    )

    model =Model(
        num_features=xtrain.shape[1],
        num_targets=ytrain.shape[1],
        num_layers=params["num_layers"],
        hidden_size=params["hidden_size"],
        dropout=params["dropout"]
    )

    model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=params["learning_rate"])
    eng = Engine(model, optimizer=optimizer, device=DEVICE)

    best_loss = np.inf
    early_stopping_iter = 10
    early_stopping_counter = 0

    for epoch in range(EPOCS):
        train_loss = eng.train(train_loader)
        valid_loss = eng.evaluate(valid_loader)
        print(f"FOLD : {fold}, EPOCH : {epoch}, TRAIN_LOSS : {train_loss}, VALIDATION_LOSS : {valid_loss}")
        if valid_loss < best_loss:
            best_loss = valid_loss
            if save_model:
                torch.save(model.state_dict(), f"Model_{fold}.bin")
        else:
            early_stopping_counter += 1

        if early_stopping_counter > early_stopping_iter:
            break
    return best_loss


# Optuna functions
def objective(trial):
    params = {
        "num_layers": trial.suggest_int("num_layers", 1, 7),
        "hidden_size": trial.suggest_int("hidden_size", 16, 2048),
        "dropout": trial.suggest_uniform("dropout", 0.1, 0.7),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 1e-3)
    }
    all_loss = []
    for f in range(5):
        temp_loss = run_training(f, params, save_model=False)
        all_loss.append(temp_loss)

    return np.mean(all_loss)


if __name__ == "__main__":
    study =optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)

    print("Best trial: ")
    trial_ = study.best_trial

    print(trial_.values)
    print(trial_.params)
    scores = 0
    for j in range(5):
        scr = run_training(j, trial_.params, save_model=True)
        scores += scr

    print(scores / 5)